<a href="https://colab.research.google.com/github/Ricardomanuel1/Maestria_Ciencia_de_Datos/blob/main/MACHINE%20LEARNING%20Y%20DEEP%20LEARNING/4_Fine_tuning_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Token Clasificación: Fine-tuning NER**

La clasificación de tokens abarca cualquier problema que pueda formularse como "atribuir una etiqueta a cada token en una oración", como por ejemplo:

- Reconocimiento de entidades nombradas (NER): busque las entidades (como personas, ubicaciones u organizaciones) en una oración. Esto se puede formular como atribuir una etiqueta a cada token teniendo una clase por entidad y una clase para "sin entidad".

- Part of Speech Tagging (POS): marque cada palabra de una oración como correspondiente a una parte particular del discurso (como sustantivo, verbo, adjetivo, etc.).

- Chunking: Encuentra los tokens que pertenecen a la misma entidad. Esta tarea (que se puede combinar con POS o NER) se puede formular como atribuir una etiqueta (generalmente B-) a cualquier token que se encuentre al comienzo de un fragmento, otra etiqueta (generalmente I-) a los tokens que se encuentran dentro de un fragmento. y una tercera etiqueta (normalmente O) para los tokens que no pertenecen a ningún fragmento.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. T

**Preparando los datos**

Se necesita un conjunto de datos adecuado para la clasificación de tokens: CoNLL-2003, que contiene noticias de Reuters.

Carga de datos

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("conll2003")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Inspección:


In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

Se puede ver que el conjunto de datos contiene etiquetas para las tres tareas que mencionadas anteriormente: NER, POS y Chunking.

Una gran diferencia con otros conjuntos de datos es que los textos de entrada no se presentan como oraciones o documentos, sino como listas de palabras (la última columna se llama tokens, pero contiene palabras en el sentido de que son entradas pre-tokenizadas que aún deben eliminarse).

El primer elemento del conjunto de entrenamiento:

In [ ]:
raw_datasets["train"][0]["tokens"]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

Como se quiere realizar el reconocimiento de entidades nombradas, se ver las etiquetas NER:

In [ ]:
raw_datasets["train"][0]["ner_tags"]

[3, 0, 7, 0, 0, 0, 7, 0, 0]

Esas son las etiquetas como números enteros listos para el entrenamiento, pero no son necesariamente útiles cuando se quiere  inspeccionar los datos.

Se puede  acceder a la correspondencia entre esos números enteros y los nombres de las etiquetas mirando el atributo de features de nuestro conjunto de datos:

In [ ]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

El tipo de elementos de la secuencia está en el atributo feature de este ner_feature, y podemos acceder a la lista de nombres mirando el atributo names de esa feature:

In [ ]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

- O significa que la palabra no corresponde a ninguna entidad.
- B-PER/I-PER significa que la palabra corresponde al comienzo de/está dentro de una entidad persona.
- B-ORG/I-ORG significa que la palabra corresponde al comienzo de/está dentro de una entidad de organización.
- B-LOC/I-LOC significa que la palabra corresponde al comienzo de/está dentro de una entidad de ubicación.
- B-MISC/I-MISC significa que la palabra corresponde al comienzo de/está dentro de una entidad miscelánea.

Decodificando:

In [ ]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

Los textos deben convertirse en IDs de token antes de que el modelo pueda entenderlos.

En este dataset se tiene entradas pre-tokenizadas. Afortunadamente, la API del tokenizador puede solucionar esto con bastante facilidad; solo necesitamos advertir al tokenizador con un flag especial.

**Tokenizer**

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
tokenizer.is_fast

True

Para tokenizar una entrada previamente tokenizada, se debe agregar `is_split_into_words=True`:

In [ ]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

**Alineamiento de etiquetas con tokens**

Como se puede ver, el tokenizador agregó los tokens especiales utilizados por el modelo ([CLS] al principio y [SEP] al final) y dejó la mayoría de las palabras intactas. La palabra lamb, sin embargo, fue dividida en dos subpalabras, la y ##mb.

Esto introduce desbalance entre nuestras entradas y las etiquetas: la lista de etiquetas tiene solo 9 elementos, mientras que la entrada ahora tiene 12 tokens. Contabilizar los tokens especiales es fácil (sabemos que están al principio y al final), pero también debemos asegurarnos de alinear todas las etiquetas con las palabras adecuadas.

Afortunadamente, debido a que se tiene un tokenizador rápido, se puede asignar fácilmente cada token a su palabra correspondiente:

In [ ]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

Se puede ahora expandir nuestra lista de etiquetas para que coincida con los tokens.

- La primera regla que se aplica es que los tokens especiales reciben una etiqueta de -100. Esto se debe a que, de forma predeterminada, -100 es un índice que se ignora en la función de pérdida usada (entropía cruzada).
- Luego, cada token recibe la misma etiqueta que el token que inició la palabra en su interior, ya que son parte de la misma entidad.
- Para tokens dentro de una palabra pero no al principio, se reemplaza la B- con I- (ya que el token no comienza la entidad):

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

Prueba con la primera oración:


In [ ]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


Como se puede ver, la función agregó el -100 para los dos tokens especiales al principio y al final, y un nuevo 0 para la palabra que se dividió en dos tokens.

Para preprocesar todo nuestro conjunto de datos, necesitamos tokenizar todas las entradas y aplicar align_labels_with_tokens() en todas las etiquetas.

Para aprovechar la velocidad del tokenizador, es mejor tokenizar muchos textos al mismo tiempo, por lo que se escribe una función que procese una lista de ejemplos y se usa el método Dataset.map() con la opción `batched=True`. Lo único que es diferente del ejemplo anterior es que la función word_ids() necesita obtener el índice del ejemplo del que queremos los ID de palabra cuando las entradas al tokenizador son listas de textos (o en nuestro caso, lista de listas de palabras), así que se agrega eso también:

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

Ahora se puede aplicar todo ese preprocesamiento de una sola vez en las otras divisiones de nuestro conjunto de datos:

In [ ]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

**Recopilación de datos (Data Colletion)**

No se puede usar simplemente un DataCollatorWithPadding porque eso solo rellena las entradas (ID de entrada, máscara de atención e ID de tipo de token). Aquí las etiquetas deben rellenarse exactamente de la misma manera que las entradas para que mantengan el mismo tamaño, usando -100 como valor para que las predicciones correspondientes se ignoren en el cálculo de la pérdida.

Todo esto se hace mediante DataCollatorForTokenClassification. Al igual que DataCollatorWithPadding, necesita el tokenizador utilizado para preprocesar las entradas:

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

Para probar esto en algunas muestras, se usa una lista de ejemplos de nuestro conjunto de entrenamiento tokenizado:

In [ ]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

Comparemos esto con las etiquetas del primer y segundo elemento de nuestro conjunto de datos:

In [ ]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]
[-100, 1, 2, -100]


**Métrica**

Para que el Trainer calcule una métrica en cada época, se necesita definir una función Compute_metrics() que tome las matrices de predicciones y clases, y devuelva un diccionario con los nombres y valores de las métricas.

Para evaluar la predicción de la clasificación de tokens se usa a`seqeval`. Para utilizar esta métrica, primero se debe instalar la biblioteca seqeval:

In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 850.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=5396dcbec77f044628108d02c4aa90c32e18ea965f06070aa7702931e219a52e
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
import evaluate

metric = evaluate.load("seqeval")

Esta métrica no se comporta como el accuracy estándar: en realidad tomará las listas de etiquetas como cadenas, no como números enteros, por lo que se necesitará decodificar completamente las predicciones y etiquetas antes de pasarlas a la métrica.

Primero, se obtiene las etiquetas para nuestro primer ejemplo de entrenamiento:

In [ ]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

Luego podemos crear predicciones falsas para ellos simplemente cambiando el valor en el índice 2:

In [ ]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'MISC': {'precision': 1.0,
  'recall': 0.5,
  'f1': 0.6666666666666666,
  'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666,
 'overall_f1': 0.8,
 'overall_accuracy': 0.8888888888888888}

Se obtiene accuracy, recall y  F1 para cada entidad por separado, así como la puntuación general. Para el cálculo de métricas, solo se mantendrá la puntuación general, pero se puede modificar la función Compute_metrics() para devolver todas las métricas que desea informar.

Esta función Compute_metrics() primero toma el argmax de los logits para convertirlos en predicciones (como es habitual, los logits y las probabilidades están en el mismo orden, por lo que no se necesita aplicar softmax).

Luego se tiene que convertir tanto las etiquetas como las predicciones de números enteros a cadenas. Se elimina todos los valores donde la etiqueta es -100, luego se pasa los resultados al método metric.compute():

In [ ]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

**Definición del Modelo**

Como se está trabajando en un problema de clasificación de tokens, se usa la clase `AutoModelForTokenClassification`. Lo principal a recordar al definir este modelo es transmitir cierta información sobre la cantidad de etiquetas que tenemos. La forma más sencilla de hacer esto es pasar ese número con el argumento `num_labels`, pero es mejor establecer las correspondencias de etiquetas correctas.

Deben establecerse mediante dos diccionarios, id2label y label2id, que contienen las asignaciones de ID a etiqueta y viceversa:

In [ ]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

Ahora se puede pasar al método AutoModelForTokenClassification.from_pretrained() y se establecerán en la configuración del modelo.

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Al definir AutoModelForSequenceClassification, al crear el modelo se emite una advertencia de que algunos pesos no se utilizaron (los del cabezal de preentrenamiento) y algunos otros pesos se inicializan aleatoriamente (los del nuevo cabezal de clasificación de tokens), y que este modelo debe ser entrenado.

 Primero se verifica que el modelo tenga la cantidad correcta de etiquetas:

In [ ]:
model.config.num_labels

9

**Fine-tuning del Modelo**

 Para definir Trainer, de debe definir los argumentos de entrenamiento:

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    "mi_ner",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
 )

Finalmente, simplemente se pasa todo al Trainer y se inicia el entrenamiento:

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.076500,0.065527,0.902222,0.936385,0.918986,0.982060


**Uso del Modelo**

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "mi-ner"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")